![rustronomy_dark_banner](https://github.com/smups/rustronomy/blob/main/logos/Rustronomy-fits_github_banner_dark.png?raw=true#gh-light-mode-only)
![rustronomy_light_banner](https://github.com/smups/rustronomy/blob/main/logos/Rustronomy-fits_github_banner_light.png#gh-dark-mode-only)
# Example 1 - reading a FITS file

Let's start by importing some dependancies:

In [37]:
:dep rustronomy-fits = {git = "https://github.com/smups/rustronomy"}
:dep ndarray = {ndarray = 0.15, features = ["blas", "approx", "std"]}
:dep dirs = {dirs = 4}

In [38]:
use rustronomy_fits as rsf;
use ndarray as nd;
use dirs;

### 1.1 - displaying the structure of a FITS file
When using Rustronomy-fits, it's often important to know what kind of data the FITS file contains _before_ you try to access any of the data contained within. This has to do with Rust being statically typed: you cant just ask for "data contained within HDU0" without knowing the type of the data.

Luckily, we can simply print the structure of a FITS file.

_Note: in this example we are using the example FITS file provided in the example folder on github_

In [39]:
use std::fs;
use std::path;

//We'll be using one of the FITS files from the tests folder
let mut path = dirs::home_dir().unwrap();
path.push("Documenten/rustronomy_project/rustronomy/rustronomy-fits/resources/tests/real_assortment/master_science0.fits");

//open and print the file (we need the mut later on)
let mut fits = rsf::fits::Fits::open(&path)?;
println!("{fits}");

 [1339.9511313479616, 1392.3042875600454, 1337.1400907286466, 1431.4846761295173, 1381.0993015542517, ..., 1377.0438753569972, 1411.6077990517654, 1389.1330903393477, 1385.7060764835862, 1389.032159743393]]
>=================================<|FITS File|>=================================
>Total File size in FITS blocks: 2678
>Number of Header-Data-Units: 2
>Contents:
>-------------------------------------------------------------------------------
>  [HDU #0]
>  Total HDU size in FITS blocks: 1
>    [Header] - size: 1, #records: 4
>    [Data] - (NO_DATA)
>-------------------------------------------------------------------------------
>  [HDU #1]
>  Total HDU size in FITS blocks: 2677
>    [Header] - size: 2, #records: 42
>    [Data] - (IMAGE) datatype: f64, shape: (956,1007), size: 2675
>===============================================================================




from the print-out we can see that we have a `f64` Image in the second header-data-unit. We can get more information about the Image by printing its header:

In [40]:
println!("{}", fits.get_hdu(1).unwrap().get_header());

>================================<|FITS Header|>================================
>Size in FITS blocks: 2
>  [CCDYPIXE] : 0.00000000
>  [EXTNAME] : extension1
>  [PCOUNT] : 0
>  [MAXBIAS] : 65535.00000000
>  [DASCHAN] : 4
>  [WINNO] : 4
>  [CRPIX2] : 1.00000000
>  [CUNIT1] : pixel
>  [CCDTYPE] : EEV42-80
>  [CUNIT2] : pixel
>  [CD1_1] : 1.00000000
>  [CRPIX1] : 1.00000000
>  [RTDATSEC] : [668:1623,1642:2648]
>  [CRVAL2] : 1642.00000000
>  [GAIN] : 2.90000000
>  [AMPNAME] : LH
>  [BITPIX] : -64
>  [CCDXPIXE] : 0.00000000
>  [CD2_1] : 0.00000000
>  [NAXIS1] : 956
>  [CD2_2] : 1.00000000
>  [PROJP1] : 1.00000000
>  [EXTVER] : 1
>  [CHIPNAME] : A5382-1-7
>  [PV1_1] : 1.00000000
>  [PV1_2] : 0.00000000
>  [NAXIS] : 2
>  [CRVAL1] : 668.00000000
>  [INHERIT] : T
>  [] : 
>  [SATURATE] : 65535.00000000
>  [PROJP3] : 0.00000000
>  [NAXIS2] : 1007
>  [PV2_1] : 1.00000000
>  [CCDNAME] : A5382-1-7
>  [CD1_2] : 0.00000000
>  [XTENSION] : IMAGE
>  [CCDCHIP] : A5382-1-7
>  [READNOIS] : 5.80000000
>  [

### 1.2 - loading the data as a ndarray

In general, the `get_xyz()` methods in rustronomy-fits return references (unless the return type is primitive). To obtain an owned version of the data and header contained within the hdu, we have to use a `remove_hdu()` method to obtain an owned version of the hdu, which we then deconstruct using the `to_parts()` method:

In [41]:
let (header, data) = fits.remove_hdu(1).unwrap().to_parts();
println!("{fits}"); //note that hdu #1 is missing!


>=================================<|FITS File|>=================================
>Total File size in FITS blocks: 2677
>Number of Header-Data-Units: 1
>Contents:
>-------------------------------------------------------------------------------
>  [HDU #0]
>  Total HDU size in FITS blocks: 2677
>    [Header] - size: 2, #records: 42
>    [Data] - (IMAGE) datatype: f64, shape: (956,1007), size: 2675
>===============================================================================




Our next task is to convert the data that we received to something usefull. FITS files can contain various data containers, such as Images and Tables. These are represented in rsf with the `Extension` enum. `data` is one of the variants of this extension enum. In our case, we know that we have a FITS Image, which is represented as the `Image` variant. 

The actual image is contained within the enum variant. We can turn it into a ndarray by calling the `as_owned_..._array` method where we replace ... with the datatype of the image, in our case: f64:

In [42]:
let array = match data.unwrap() {
    rsf::extensions::Extension::Image(img) => img.as_owned_f64_array()?,
    _ => panic!()
};
print!("array");

thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', src/lib.rs:131:24
stack backtrace:
   0: rust_begin_unwind
             at /rustc/9d1b2106e23b1abd32fce1f17267604a5102f57a/library/std/src/panicking.rs:498:5
   1: core::panicking::panic_fmt
             at /rustc/9d1b2106e23b1abd32fce1f17267604a5102f57a/library/core/src/panicking.rs:116:14
   2: core::panicking::panic
             at /rustc/9d1b2106e23b1abd32fce1f17267604a5102f57a/library/core/src/panicking.rs:48:5
   3: run_user_code_30
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Segmentation fault.
   0: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   1: <unknown>
   2: mi_free
   3: alloc::alloc::dealloc
             at /rustc/db9d1b20bba1968c1ec1fc49616d4742c1725b4b/library/alloc/src/alloc.rs:105:14
      <alloc::alloc::Global as core::alloc

Error: Child process terminated with status: signal: 6 (core dumped)